Mediapipe Pose Front Camera

In [5]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import time
import os
from playsound import playsound

# -----------------------------
# Landmark indices (stable)
# -----------------------------
LEFT_EAR = 7
RIGHT_EAR = 8
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12

# -----------------------------
# Utility functions
# -----------------------------
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

def draw_angle(img, p1, p2, p3, angle, color):
    cv2.line(img, p1, p2, color, 2)
    cv2.line(img, p2, p3, color, 2)
    cv2.putText(img, f"{angle:.1f}", (p2[0]+5, p2[1]-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# -----------------------------
# Config
# -----------------------------
sound_file = "alert.wav"  # Sound to play on poor posture
alert_cooldown = 5        # seconds between alerts

is_calibrated = False
calibration_frames = 0
calib_shoulder = []
calib_neck = []
last_alert_time = 0

# -----------------------------
# MediaPipe PoseLandmarker setup
# -----------------------------
BaseOptions = python.BaseOptions
PoseLandmarker = vision.PoseLandmarker
PoseLandmarkerOptions = vision.PoseLandmarkerOptions
RunningMode = vision.RunningMode

model_path = "pose_landmarker_lite.task"
assert os.path.exists(model_path), "Model file 'pose_landmarker_lite.task' not found!"

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=RunningMode.VIDEO,
    num_poses=1,
    min_pose_detection_confidence=0.5,
    min_pose_presence_confidence=0.5,
    min_tracking_confidence=0.5,
)

pose = PoseLandmarker.create_from_options(options)

# -----------------------------
# Webcam
# -----------------------------
cap = cv2.VideoCapture(0)

# -----------------------------
# Main loop
# -----------------------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    timestamp_ms = int(time.time()*1000)
    results = pose.detect_for_video(mp_image, timestamp_ms)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks[0]

        def lm(i):
            return int(landmarks[i].x * w), int(landmarks[i].y * h)

        left_shoulder = lm(LEFT_SHOULDER)
        right_shoulder = lm(RIGHT_SHOULDER)
        left_ear = lm(LEFT_EAR)
        right_ear = lm(RIGHT_EAR)

        mid_shoulder = ((left_shoulder[0]+right_shoulder[0])//2,
                        (left_shoulder[1]+right_shoulder[1])//2)

        # Calculate angles for both sides
        shoulder_angle_left = calculate_angle(left_shoulder, mid_shoulder, (mid_shoulder[0],0))
        shoulder_angle_right = calculate_angle(right_shoulder, mid_shoulder, (mid_shoulder[0],0))
        neck_angle_left = calculate_angle(left_ear, left_shoulder, (left_shoulder[0],0))
        neck_angle_right = calculate_angle(right_ear, right_shoulder, (right_shoulder[0],0))

        # -----------------------------
        # Calibration (first 30 frames)
        # -----------------------------
        if not is_calibrated and calibration_frames < 30:
            calib_shoulder.append(shoulder_angle_left)
            calib_shoulder.append(shoulder_angle_right)
            calib_neck.append(neck_angle_left)
            calib_neck.append(neck_angle_right)
            calibration_frames += 1
            cv2.putText(frame, f"Calibrating {calibration_frames}/30",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        elif not is_calibrated:
            shoulder_thresh = np.mean(calib_shoulder) - 10
            neck_thresh = np.mean(calib_neck) - 10
            is_calibrated = True
            print(f"Calibration complete. Shoulder threshold: {shoulder_thresh:.1f}, Neck threshold: {neck_thresh:.1f}")

        # -----------------------------
        # Draw landmarks
        # -----------------------------
        for pt in landmarks:
            cx, cy = int(pt.x * w), int(pt.y * h)
            cv2.circle(frame, (cx, cy), 3, (0,255,0), -1)

        # Draw angles
        draw_angle(frame, left_shoulder, mid_shoulder, (mid_shoulder[0],0), shoulder_angle_left, (255,0,0))
        draw_angle(frame, right_shoulder, mid_shoulder, (mid_shoulder[0],0), shoulder_angle_right, (0,0,255))
        draw_angle(frame, left_ear, left_shoulder, (left_shoulder[0],0), neck_angle_left, (0,255,0))
        draw_angle(frame, right_ear, right_shoulder, (right_shoulder[0],0), neck_angle_right, (0,255,255))

        # -----------------------------
        # Feedback (trigger if either side off threshold)
        # -----------------------------
        if is_calibrated:
            now = time.time()
            poor_posture = (
                shoulder_angle_left < shoulder_thresh or
                shoulder_angle_right < shoulder_thresh or
                neck_angle_left < neck_thresh or
                neck_angle_right < neck_thresh
            )

            if poor_posture:
                status = "Poor Posture"
                color = (0,0,255)
                if now - last_alert_time > alert_cooldown:
                    print("Poor posture detected!")
                    if os.path.exists(sound_file):
                        playsound(sound_file)
                    last_alert_time = now
            else:
                status = "Good Posture"
                color = (0,255,0)

            # Display status and angles
            cv2.putText(frame, status, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2)
            cv2.putText(frame, f"Shoulder L/R: {shoulder_angle_left:.1f}/{shoulder_angle_right:.1f}",
                        (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255),1)
            cv2.putText(frame, f"Neck L/R: {neck_angle_left:.1f}/{neck_angle_right:.1f}",
                        (10,90), cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),1)

    cv2.imshow("Posture Corrector", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1768018974.420657  181838 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3 Max
W0000 00:00:1768018974.482737  181840 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768018974.489196  181840 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Calibration complete. Shoulder threshold: 80.0, Neck threshold: 18.6
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor posture detected!
Poor postur

KeyboardInterrupt: 